## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-11-02-17-17 +0000,nypost,Disturbing video showed moment victim shouted ...,https://nypost.com/2025/09/10/us-news/disturbi...
1,2025-09-11-02-16-26 +0000,startribune,Minnesota politicians condemn violence and off...,https://www.startribune.com/minnesota-politici...
2,2025-09-11-02-10-42 +0000,nyt,Republicans Block Schumer’s Effort to Force Vo...,https://www.nytimes.com/2025/09/10/us/politics...
3,2025-09-11-02-08-06 +0000,nyt,"At the White House, Trump’s Grief and Shock Tu...",https://www.nytimes.com/2025/09/10/us/politics...
4,2025-09-11-02-00-04 +0000,startribune,Conservative activist Charlie Kirk assassinate...,https://www.startribune.com/turning-point-usa-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2300/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
31,trump,43
21,kirk,39
20,charlie,38
108,new,24
64,was,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
37,2025-09-11-00-08-00 +0000,wsj,Charlie Kirk was a provocative conservative in...,https://www.wsj.com/politics/charlie-kirk-turn...,176
129,2025-09-10-20-05-36 +0000,latimes,Charlie Kirk was a force with young conservati...,https://www.latimes.com/politics/story/2025-09...,149
105,2025-09-10-20-59-57 +0000,bbc,Charlie Kirk has died after being shot at Utah...,https://www.bbc.com/news/articles/c0r0q1x74qqo...,147
116,2025-09-10-20-36-13 +0000,nyt,Trump and Leaders Across Political Spectrum Re...,https://www.nytimes.com/2025/09/10/us/charlie-...,146
31,2025-09-11-00-16-47 +0000,nyt,"Charlie Kirk, Right-Wing Force and a Close Tru...",https://www.nytimes.com/2025/09/10/us/politics...,141


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
37,176,2025-09-11-00-08-00 +0000,wsj,Charlie Kirk was a provocative conservative in...,https://www.wsj.com/politics/charlie-kirk-turn...
203,93,2025-09-10-17-08-00 +0000,wsj,Former Vice President Kamala Harris said in a ...,https://www.wsj.com/politics/elections/harris-...
115,67,2025-09-10-20-37-00 +0000,wsj,Nearly half a day after NATO ordered jet fight...,https://www.wsj.com/politics/national-security...
259,59,2025-09-10-13-57-09 +0000,wapo,Israel escalates attacks on Gaza City as Qatar...,https://www.washingtonpost.com/world/2025/09/1...
304,56,2025-09-10-10-00-00 +0000,nypost,Lisa Cook should be kicked out of Fed’s Septem...,https://nypost.com/2025/09/10/us-news/lisa-coo...
245,45,2025-09-10-15-04-00 +0000,wsj,If the Supreme Court rules in favor of the pre...,https://www.wsj.com/economy/trade/tariff-case-...
28,44,2025-09-11-00-22-00 +0000,wsj,Senate Republicans voted down a measure to rel...,https://www.wsj.com/politics/policy/senate-rep...
330,44,2025-09-10-08-17-33 +0000,nypost,South Korea sends plane to US to bring back wo...,https://nypost.com/2025/09/10/world-news/south...
333,41,2025-09-10-08-07-20 +0000,nyt,Before and after photos show how protests dama...,https://www.nytimes.com/live/2025/09/10/world/...
56,37,2025-09-10-23-12-16 +0000,latimes,"Woman threw up, so Uber driver kicked her out....",https://www.latimes.com/california/story/2025-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
